# Day 11

## Imports and data loading

In [ ]:
from utils import get_input, load_data

day = 11


In [ ]:
get_input(day)


In [ ]:
data = load_data(day, list_type="line", number=False)
test_data = [
    "5483143223",
    "2745854711",
    "5264556173",
    "6141336146",
    "6357385478",
    "4167524645",
    "2176841721",
    "6882881134",
    "4846848554",
    "5283751526",
]
test_answer_1 = 1656
test_answer_2 = 195


## Part one

In [ ]:
# Let's try this with numpy
import numpy as np


class Cave:
    def __init__(self, data):
        """Create an array of arrays from the data, plus a 10x10 matrix of False."""
        self.matrix = np.array([[int(i) for i in row] for row in data])
        self.flash_matrix = np.zeros((10, 10), dtype=bool)
        self.flash_count = 0
        self.time = 0
        self.all_flash = None

    def __repr__(self):
        """Print the arrays out neatly"""
        rows = ["".join([str(r).rjust(3, " ") for r in row]) for row in self.matrix]
        return "\n".join(rows)

    def step(self):
        """Increase values by 1, increase neighbours over 9, count flashes and reset."""
        self.matrix += 1
        # Run the iterative routine to look for elements over 9 and increase neighbours
        self.increase()
        # Flash and reset everything over 9
        self.flash()
        self.time += 1
        # For part 2 - flag if the whole matrix has been reset to 0
        if np.array_equal(self.matrix, np.zeros((10, 10))):
            self.all_flash = self.time

    def flash(self):
        """Count number of elements over 9, then reset them to 0."""
        self.flash_count += np.count_nonzero(self.matrix > 9)
        self.matrix[self.matrix > 9] = 0

    def increase_neighbours(self, y, x):
        """Keep (y, x) the same but increase its 8 neighbours by 1."""
        # Use mins and maxes to adjust for matrix edges
        self.matrix[max(y - 1, 0) : min(y + 2, 10), max(x - 1, 0) : min(x + 2, 10)] += 1
        # Previous line also increases (y, x), so this one reduces it again
        self.matrix[y, x] -= 1
        # Note that this element has flashed
        self.flash_matrix[y, x] = True

    def increase(self):
        """increase_neighbours on anything over 9, and continue until no more to flash."""
        # Continue until all the elements over 9 have already flashed
        while (self.matrix > 9).sum() - self.flash_matrix[self.matrix > 9].sum() > 0:
            # Set up an iterator that allows writing back to the matrix
            iter = np.nditer(self.matrix, flags=["multi_index"], op_flags=["readwrite"])
            for point in iter:
                if (
                    # Only flash if that element hasn't already flashed
                    point > 9
                    and not self.flash_matrix[iter.multi_index[0], iter.multi_index[1]]
                ):
                    self.increase_neighbours(iter.multi_index[0], iter.multi_index[1])
        # Remember to reset the flash_matrix after each step
        self.flash_matrix = np.zeros((10, 10), dtype=bool)


In [ ]:
test_cave = Cave(test_data)
cave = Cave(data)
for i in range(100):
    test_cave.step()
    cave.step()
assert test_cave.flash_count == test_answer_1

cave.flash_count


## Part two

In [ ]:
test_cave = Cave(test_data)
cave = Cave(data)

while not test_cave.all_flash:
    test_cave.step()

assert test_cave.all_flash == test_answer_2

while not cave.all_flash:
    cave.step()

print(cave.all_flash)
